In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import cv2, time
from datetime import datetime
import slackweb
import os

#proxy認証
#os.environ["HTTP_PROXY"] = "proxy_adress"
#os.environ["HTTPS_PROXY"] = "proxy_adress"

# 正面の顔検出用
cascPath = '-----Your_Path-----/lbpcascades/lbpcascade_frontalface.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

#Slackに通知を飛ばす
#slack = slackweb.Slack(url="<https://hooks.slack.com/services/-----Your_adress-----")

#MJPEGStream
URL = "-----raspberry_address------/?action=stream"
s_video = cv2.VideoCapture(URL)

while True:
  ret, img = s_video.read()

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.equalizeHist( gray )
  faces = faceCascade.detectMultiScale(gray, 1.1, 3, 0, (10, 10))

  for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
      time = datetime.now().strftime('%Y%m%d%H%M%S')
      # imgフォルダに格納
      path = '.\\\\img\\\\' + time + '.jpg'
      cut_face = img[y:y+h,x:x+w,:]
      cut_face_resize = cv2.resize(cut_face,(64,64))
      cv2.imwrite(path, cut_face_resize)
      # slackに通知
      # slack.notify(text="メッセージ")
    
  cv2.imshow("Face-Detect-Video",img)
  key = cv2.waitKey(1) & 0xff
  if key == ord('q'): break